In [1]:
import ipywidgets as widgets
from PIL import Image
import io

error_images_ejemplo_1 = [
    '../Basico/imagenes/error/error_image_1.png',
    '../Basico/imagenes/error/error_image_2.png',
    '../Basico/imagenes/error/error_image_3.png',
    '../Basico/imagenes/error/error_image_4.png',
    '../Basico/imagenes/error/error_image_5.png'
]

def create_button(description, color, font_size, font_weight, width='auto'):
    button = widgets.Button(
        description=description,
        layout=widgets.Layout(width=width, padding='2px', margin='5px'),
        style={'button_color': color, 'font_weight': font_weight, 'font_size': font_size, 'text_align': 'center'}
    )
    return button

def display_image(image_path, format='png'):
    with open(image_path, "rb") as f:
        img = Image.open(f)
        img_byte_array = io.BytesIO()
        img.save(img_byte_array, format=format)
        img_data = img_byte_array.getvalue()
    return widgets.Image(value=img_data, format=format)

class BaseQuizApp:
    def __init__(self, quiz_data):
        self.quiz_data = quiz_data
        self.current_question_index = 0
        self.error_count = 0
        self.output_box = widgets.Output()
        self.create_quiz()
        self.display()

    def create_quiz(self):
        pass

    def display(self):
        with self.output_box:
            self.output_box.clear_output()
            display(self.question_label)
            display(self.image_box)  
            display(self.options_box)

class QuizApp_ejemplo_1(BaseQuizApp):
    def __init__(self, quiz_data):
        super().__init__(quiz_data)

    def create_quiz(self):
        self.output_box.clear_output()
        self.option_buttons = []

        if self.current_question_index < len(self.quiz_data):
            quiz = self.quiz_data[self.current_question_index]

            self.question_label = widgets.HTML(
                value=f"<h2 style='color: #0056b3; text-align: center;'>{quiz['question']}</h2>"
            )

            self.option_buttons = [
                create_button(option, '#e8ebe9', '14px', 'bold') for option in quiz['options']
            ]

            for button in self.option_buttons:
                button.on_click(lambda b, opt=button.description: self.on_button_clicked(opt, quiz['correct_option']))

            with open('../Basico/imagenes/base/base.png', 'rb') as f:
                self.image_widget = widgets.Image(
                    value=f.read(),
                    format='png', 
                    layout=widgets.Layout(max_width='700px', max_height='700px', object_fit='contain')
                )
            self.image_box = widgets.HBox([self.image_widget], layout=widgets.Layout(justify_content='center'))

            self.options_box = widgets.VBox(self.option_buttons, layout=widgets.Layout(align_items='center'))

        else:
            self.question_label = widgets.HTML(
                value="<h2 style='color: #28a745; text-align: center;'>¡Bien hecho!</h2>"
            )
            self.options_box = widgets.VBox([])
            self.image_box = widgets.VBox([])

    def on_button_clicked(self, option, correct_option):
        if option == correct_option:
            self.show_message("Correcto", "¡Buena!")
            self.next_question()
        else:
            self.error_count += 1
            message = self.get_error_message()
            self.show_error_message(message)

    def get_error_message(self):
        if self.error_count < 2:
            return "Whops, esa no era la respuesta correcta. Prueba otra vez."
        elif self.error_count < 4:
            return "¿Estás seguro de que has atendido a la explicación?"
        elif self.error_count < 6:
            return "Estás haciendo al búho llorar."
        elif self.error_count < 7:
            return "¿Te diviertes?"
        else:
            return "..."

    def show_message(self, title, message):
        with self.output_box:
            self.output_box.clear_output()
            display(widgets.HTML(f"<h3 style='text-align: center;'>{title}: {message}</h3>"))

    def show_error_message(self, message):
        with self.output_box:
            self.output_box.clear_output()
    
            error_img_path = self.get_error_image_path()
    
            with open(error_img_path, "rb") as file:
                error_img = widgets.Image(
                    value=file.read(),
                    format='png',  
                    layout=widgets.Layout(max_width='500px', max_height='400px', object_fit='contain')
                )
    
            error_button = create_button("Venga voy", '#f44336', '14px', 'bold')
            error_button.on_click(self.retry_current_question)
    
            display(widgets.VBox([
                widgets.HBox([error_img], layout=widgets.Layout(justify_content='center')),
                widgets.HTML(f"<h4 style='text-align: center;'>{message}</h4>"),
                error_button
            ], layout=widgets.Layout(align_items='center')))

    def retry_current_question(self, b):
        self.create_quiz()
        self.display()

    def get_error_image_path(self):
        index = min(self.error_count - 1, len(error_images_ejemplo_1) - 1)
        return error_images_ejemplo_1[index]

    def next_question(self):
        self.current_question_index += 1
        self.error_count = 0
        self.create_quiz()
        self.display()

def run_quiz(quiz_data):
    quiz_app = QuizApp_ejemplo_1(quiz_data)
    display(quiz_app.output_box)


<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
<style>
    body {
        font-family: Arial, sans-serif;
        background-color: #000000;
        margin: 0;
        padding: 0;
    }
    .content {
        max-width: 90%;
        width: 150ch;
        margin: 0 auto;
        padding: 20px;
        background-color: rgba(255, 255, 255, 0.8); 
        box-shadow: 0 0 20px rgba(0, 0, 0, 0.5);
        border-top: 1px solid rgba(0, 0, 0, 0.5);
    }
    h1, h2, h3 {
        color: #333;
    }
    a {
        color: #1a73e8;
        text-decoration: none;
    }
    a:hover {
        text-decoration: underline;
    }
    em {
        font-style: italic;
    }
    strong {
        font-weight: bold;
    }
    @media (max-width: 1290px) {
        .content {
            max-width: 80%; 
            padding: 20px;
            width: 120ch; 
        }
    }
    @media (max-width: 480px) {
        .content {
            max-width: 100%;
            padding: 20px; 
        }
    }
</style>
</head>
<body>
    <div class="content">
       
<h1 id="paradigmas-de-programaci-n">Paradigmas de programación</h1>
<p>Como dice el dicho, una línea de código vale más que mil palabras. ¡Vamos a comparar rápidamente las diferencias entre un desarrollo en un lenguaje imperativo como Python y uno declarativo como Prolog!<BR></p>
<h3 id="desarrollo-imperativo">Desarrollo imperativo</h3>
<p>Podríamos plantear un metáfora sobre la manera de operar de diferentes personas frente a una tarea, como podría ser hacer una tarta.<br>
Primero vamos a pensar en una persona &quot;normal&quot;. Cuando se le asigna una tarea como esta, definirá una serie de pasos de la A a la G e irá realizando cada uno de ellos de manera secuencial, sin variar el orden bajo ninguna circunstancia.<br></p>
<pre><code>A- Batir huevos.
B- Mezclar los ingredientes secos.
C- Mezclar los ingredientes húmedos.
D- Echar la mezcla en un molde.
E- Meter la tarta en el horno a <span class="hljs-number">180</span>º.
F- Sacar la tarta a los <span class="hljs-number">20</span> minutos.
G- Decorar y servir.
</code></pre><p>Ahora bien, ¿qué ocurre si no hay huevos? ¿O si al mezclar los ingredientes nos falta leche? ¿Y si el horno de repente no funciona? Esto es lo que conocemos como <strong>excepciones</strong>. Posibilidades no contempladas en el flujo original.<br>
En este caso, la excepción se podrá tratar de varias maneras una vez ocurra, pidiendo ingredientes al vecino, sustituyendo elementos, yendo a comprar o tirando con enfado la tarta a medio hacer por la ventana y pasando del tema. El problema fundamental de esta aproximación consiste en que hay que tener muy claras todas las posibles desviaciones que pueden aparecer durante la ejecución, ya que una excepción no contemplada nos puede generar que el programa simplemente deje de funcionar.<br></p>
<p>Sin embargo, si ponemos a una persona con TDAH a hacer la misma tarta, esta persona primero revisará que tiene todo lo que necesita para realizar la tarta, negándose siquiera a empezar en caso de que le faltara la cosa más nimia, y en caso de tener todo disponible procederá a preparar la tarta.<br></p>
<p>Llevando esto a código, si nosotros quisiéramos hacer un sandwich, podríamos definir una función en Python de la siguiente manera:<br></p>
<pre><code><span class="hljs-class"><span class="hljs-keyword">class</span> <span class="hljs-title">Sandwich</span>:  </span>
    <span class="hljs-function"><span class="hljs-keyword">def</span> <span class="hljs-title">__init__</span><span class="hljs-params">(<span class="hljs-keyword">self</span>, pan1, pan2, relleno)</span></span>:  
        <span class="hljs-keyword">self</span>.pan1 = pan1  
        <span class="hljs-keyword">self</span>.pan2 = pan2  
        <span class="hljs-keyword">self</span>.relleno = relleno<br>
    <span class="hljs-function"><span class="hljs-keyword">def</span> <span class="hljs-title">__str__</span><span class="hljs-params">(<span class="hljs-keyword">self</span>)</span></span>:
        <span class="hljs-keyword">return</span> f<span class="hljs-string">"{self.pan1} {self.relleno} {self.pan2}"</span><br>
    <span class="hljs-function"><span class="hljs-keyword">def</span> <span class="hljs-title">hacer_sandwich</span><span class="hljs-params">(pan1, pan2, relleno)</span></span>:
        sandwich = Sandwich(pan1, pan2, relleno)
        <span class="hljs-keyword">return</span> f<span class="hljs-string">"Sándwich pa ti, campeón: {sandwich}"</span><br>
</code></pre><p>Que tendría el siguiente output:<br></p>
</body>
</html>


In [10]:
import ipywidgets as widgets
from IPython.display import display

class Sandwich:
    def __init__(self, pan1, pan2, relleno):
        self.pan1 = pan1
        self.pan2 = pan2
        self.relleno = relleno
    
    def __str__(self):
        return f"{self.pan1} {self.relleno} {self.pan2}"

def hacer_sandwich(pan1, pan2, relleno):
    sandwich = Sandwich(pan1, pan2, relleno)
    return f"Sándwich pa ti, campeón: {sandwich}"

out1 = widgets.Output()

def on_button_clicked(_):
    with out1:
        out1.clear_output()
        result = hacer_sandwich("Pan Blanco", "Pan Integral", "Jamón y Queso")
        print(result)

button = widgets.Button(
    description='¡Haz click para hacer un sándwich!',
    icon='hand-o-up',
    layout=widgets.Layout(width='auto')
)

button.on_click(on_button_clicked)

centered_button = widgets.HBox([button], layout=widgets.Layout(justify_content='center'))
centered_output = widgets.HBox([out1], layout=widgets.Layout(justify_content='center'))
display(widgets.VBox([centered_button, centered_output], layout=widgets.Layout(align_items='center')))


<!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">
       
<p>El flujo es tremendamente sencillo. El programa espera que se le proporcionen dos panes y un relleno para el sándwich, y una vez se le da esto simplemente encierra el relleno entre los dos panes y lo presenta al usuario.<br>
¿Cuál es el problema aquí? Pues podemos hacer una prueba. ¿Qué ocurriría si se nos antoja hacer un sándwich con dos alpargatas y medio kilo de arena de gato, dos cubos de basura y un calcetín sudao o lo que se nos ocurra?<BR></p>
</body>
</html>


In [11]:
import ipywidgets as widgets
from IPython.display import display

class Sandwich:
    def __init__(self, pan1, pan2, relleno):
        self.pan1 = pan1
        self.pan2 = pan2
        self.relleno = relleno
    
    def __str__(self):
        return f"{self.pan1} {self.relleno} {self.pan2}"

def hacer_sandwich(pan1, pan2, relleno):
    sandwich = Sandwich(pan1, pan2, relleno)
    return f"Sándwich pa ti, campeón: {sandwich}"

out2 = widgets.Output()

pan1_input = widgets.Text(
    placeholder='Escribe el tipo de pan 1...',
    description='Pan 1:',
    disabled=False,
    continuous_update=False
)

pan2_input = widgets.Text(
    placeholder='Escribe el tipo de pan 2...',
    description='Pan 2:',
    disabled=False,
    continuous_update=False
)

relleno_input = widgets.Text(
    placeholder='Escribe el relleno...',
    description='Relleno:',
    disabled=False,
    continuous_update=False
)

input_container = widgets.VBox()

step = 0

def on_key_press(event):
    global step
    if event['name'] == 'value' and event['new'] != '':
        if step == 1 and event['owner'] is pan1_input:
            input_container.children = [pan2_input]
            step += 1
        elif step == 2 and event['owner'] is pan2_input:
            input_container.children = [relleno_input]
            step += 1
        elif step == 3 and event['owner'] is relleno_input:
            input_container.children = []
            pan1 = pan1_input.value
            pan2 = pan2_input.value
            relleno = relleno_input.value
            step = 0
            result = hacer_sandwich(pan1, pan2, relleno)
            with out2:
                out2.clear_output()
                print(result)

def on_button_clicked(_):
    global step
    step = 1
    input_container.children = [pan1_input]

pan1_input.observe(on_key_press, names='value')
pan2_input.observe(on_key_press, names='value')
relleno_input.observe(on_key_press, names='value')

button = widgets.Button(
    description='¡Haz click para hacer un sándwich!',
    icon='hand-o-up',
    layout=widgets.Layout(width='auto')
)

button.on_click(on_button_clicked)

centered_button = widgets.HBox([button], layout=widgets.Layout(justify_content='center'))
centered_output = widgets.HBox([out2], layout=widgets.Layout(justify_content='center'))

display(widgets.VBox([centered_button, input_container, centered_output], layout=widgets.Layout(align_items='center')))


<!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">
       <p>Pues efectivamente, para nuestra desgracia tendríamos el sándwich menos apetecible de la historia sin mayor alternativa. <br>
Claramente, una de las cuestiones a tratar aquí es establecer que, por ejemplo, el pan sea pan y el relleno sea comestible, excluyendo cualquier otra entrada que nos proporcionen en caso de que no se cumplan estas condiciones, lo cual nos lleva al eterno
problema de la programación. El cliente no siempre tendrá la razón, pero siempre encontrará maneras de reventar el programa que al desarrollador no se le habrían ocurrido en sueños.<br></p>
<h3 id="desarrollo-declarativo">Desarrollo declarativo</h3>
<p>Sin embargo, si quisiéramos hacer lo mismo con Prolog, el código sería el siguiente: <br>
    <pre><code>pan(rebanada).
relleno(jamon_y_queso).
relleno(salchicha_y_queso).
relleno(huevo_y_bacon).
relleno(carne_picada).
</code></pre>
<pre><code>
hacer_sandwich(Sandwich) :-
    <span class="hljs-function"><span class="hljs-title">rebanada_de_pan</span><span class="hljs-params">(Rebanada1)</span></span>,    
    <span class="hljs-function"><span class="hljs-title">rebanada_de_pan</span><span class="hljs-params">(Rebanada2)</span></span>,
    <span class="hljs-function"><span class="hljs-title">relleno</span><span class="hljs-params">(Relleno)</span></span>,    
    Sandwich = [Rebanada1, Relleno, Rebanada2].<br>
</code></pre><p>Esto, al ejecutarse nos devuelve lo siguiente:</p>

</body>
</html>


In [5]:
import ipywidgets as widgets
from IPython.display import display
from swiplserver import *

out_prolog = widgets.Output()

def run_prolog_query(_):
    with out_prolog:
        out_prolog.clear_output()
        with PrologMQI() as mqi:
            with mqi.create_thread() as prolog_thread:
                result = prolog_thread.query("set_prolog_flag(encoding,utf8).")
                try:
                    result = prolog_thread.query('consult("ejemplos/sandwich.pl")')
                    prolog_thread.query_async("hacer_sandwich(X).", find_all=False)        
                    while True:
                        result = prolog_thread.query_async_result()
                        if result is None:
                            break
                        else:
                            print(result)        
                except Exception as e:
                    print(e)

button_prolog = widgets.Button(
    description='Deja que Prolog haga su cosa',
    icon='play',
    layout=widgets.Layout(width='auto')
)

button_prolog.on_click(run_prolog_query)

centered_button_prolog = widgets.HBox([button_prolog], layout=widgets.Layout(justify_content='center'))
centered_output_prolog = widgets.HBox([out_prolog], layout=widgets.Layout(justify_content='center'))

display(widgets.VBox([centered_button_prolog, centered_output_prolog], layout=widgets.Layout(align_items='center')))


[{'X': ['rebanada', 'jamon_y_queso', 'rebanada']}]
[{'X': ['rebanada', 'salchicha_y_queso', 'rebanada']}]
[{'X': ['rebanada', 'huevo_y_bacon', 'rebanada']}]
[{'X': ['rebanada', 'carne_picada', 'rebanada']}]


<!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">
       <p>Una cosa a recalcar con Prolog es que hay muchas maneras de interactuar con un programa escrito en Prolog.<br>
Nosotros podemos realizar una consulta a este programa sin especificarle nada, y en este caso Prolog nos construirá primorosamente un sándwich para nuestro deleite.<br>
No obstante, también podemos optar por especificarle qué cosas queremos en nuestro sándwich, y Prolog intentará hacer un sándwich a partir de esto unificando lo especificado con los hechos en nuestra base de conocimiento y dándonos el ok si puede hacer un sandwich con lo que le proporcionamos, a diferencia de Python que cogerá los ingredientes, los pondrá uno encima del otro y lo llamará sándwich sin atender a si tiene sentido o no.<br>
Es decir, nuestro enfoque debe ser ver cómo interacionan los elementos entre sí a lo largo del ciclo de vida del programa, no establecer una serie de pasos a cumplir y programar posibles desviaciones y excepciones para este patrón.<br>
Nuestro trabajo consistirá en definir las relaciones de nuestro dominio, en lugar de crear un flujo y tratar la infinidad de errores que pueden surgir por el camino.<br>
Cualquier consulta que tenga como parámetros cualquier elemento que no haya sido considerado por el experto que ha establecido las reglas de nuestro dominio nos devolverá un Falso como resultado, en lugar de tener una excepción.<br></p>

</body>
</html>


In [1]:
import ipywidgets as widgets
from IPython.display import display
from swiplserver import PrologMQI

out3 = widgets.Output()

pan1_input2 = widgets.Text(
    placeholder='Escribe el tipo de pan 1...',
    description='Pan 1:',
    disabled=False,
    continuous_update=False
)

pan2_input2 = widgets.Text(
    placeholder='Escribe el tipo de pan 2...',
    description='Pan 2:',
    disabled=False,
    continuous_update=False
)

relleno_input2 = widgets.Text(
    placeholder='Escribe el relleno...',
    description='Relleno:',
    disabled=False,
    continuous_update=False
)

input_container2 = widgets.VBox()

step1 = 0

def on_key_press(event):
    global step1
    if event['name'] == 'value' and event['new'] != '':
        if step1 == 1 and event['owner'] is pan1_input2:
            input_container2.children = [pan2_input2]
            step1 += 1
        elif step1 == 2 and event['owner'] is pan2_input2:
            input_container2.children = [relleno_input2]
            step1 += 1
        elif step1 == 3 and event['owner'] is relleno_input2:
            input_container2.children = []
            step1 = 0
            
            pan1 = pan1_input2.value
            pan2 = pan2_input2.value
            relleno = relleno_input2.value
            
            with PrologMQI() as mqi:
                with mqi.create_thread() as prolog_thread:
                    prolog_thread.query("set_prolog_flag(encoding,utf8).")
                    try:
                        prolog_thread.query("consult(\"ejemplos/sandwich.pl\")")
                        query = f"hacer_sandwich({pan1},{pan2},{relleno})."
                        prolog_thread.query_async(query, find_all=True)
                        result = prolog_thread.query_async_result()
                        if result is False:
                            with out3:
                                out3.clear_output()
                                print("QUE CLASE DE SANDWICH ME ESTAS ARMANDO, PEDAZO DE PSICOPATA")
                        else:
                            with out3:
                                out3.clear_output()
                                print(result)
                    except Exception as e:
                        with out3:
                            out3.clear_output()
                            print(f"Error: {e}")

def on_button_clicked(_):
    global step1
    step1 = 1
    input_container2.children = [pan1_input2]

pan1_input2.observe(on_key_press, names='value')
pan2_input2.observe(on_key_press, names='value')
relleno_input2.observe(on_key_press, names='value')

button = widgets.Button(
    description='¡Haz click para hacer un sándwich!',
    icon='hand-o-up',
    layout=widgets.Layout(width='auto')
)

button.on_click(on_button_clicked)

centered_button = widgets.HBox([button], layout=widgets.Layout(justify_content='center'))
centered_output = widgets.HBox([out3], layout=widgets.Layout(justify_content='center'))

display(widgets.VBox([centered_button, input_container2, centered_output], layout=widgets.Layout(align_items='center')))


<!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">
       <h1 id="cambio-de-paradigma-">Cambio de Paradigma.</h1>
<p>Ahora mismo estoy oyendo lo que piensas.<br>
<strong>&quot;Vale si, esto es todo muy bonito, ¿pero qué carajo es un paradigma y cómo dejo de pensar en imperativo?&quot;</strong> <br>
Y tienes toda la razón, quizás me he calentado un poquillo. En el mundo de la programación llamamos &quot;Paradigmas&quot;  a los distintos enfoques sobre cómo resolver un problema. Son <strong>marcos</strong> sobre los que desarrollar la solución, cada uno con una serie de convenciones que los definen. <br>
Cada paradigma ofrece una manera diferente de pensar y resolver problemas en programación, y el uso de uno u otro puede depender del tipo de problema que se desea abordar, de las preferencias personales o de desarrollo.<br>
Por desgracia, todo lo que hayas aprendido de programación (imperativa u orientada a objetos) hasta ahora no solo no te servirá para Prolog, muy probablemente que te supondrá un obstáculo: Intentarás resolver los problema en Prolog “imperativamente” y fracasarás estrepitosamente (mientras se empiezan a oir tus lloros, oraciones y gritos y tu teclado sufre las consecuencias de tus decisiones).<br>
En un lenguaje imperativo, tú conduces (controlas la ejecución), decidiendo cómo llegar a tu destino, En Prolog, conduce Prolog (su motor de inferencia) y a ti te toca hacer toda la hoja de ruta, asegurándote de que el tráiler con una bomba nuclear lógica no se cae por un terraplén.<br>
El paradigma imperativo propone <strong>cómo</strong> se deben hacer las cosas. Nosotros determinamos el flujo de ejecución, teniendo el control y definiendo el mismo mediante instrucciones y estructuras de control (If/else, bucles For, etc...) y es nuestra responsabilidad que éste se cumpla.<br>
El paradigma declarativo propone <strong>qué</strong> se quiere conseguir, definiendo el resultado y abstrayéndonos de cómo se consigue, siendo un <strong>motor de ejecución</strong> el que lleva a cabo la ejecución sin intervención directa ni control explícito del flujo de ejecución por nuestra parte. Uno de los primeros shocks aquí es que primero necesitamos comprender el funcionamiento del motor de ejecución (en caso de Prolog, el motor de inferencia) para poder plantear las reglas que definirán nuestro dominio.<br>
Puede que no hayas oído hablar de paradigmas en profundidad, pero con total seguridad tienes experiencia tanto en lenguajes imperativos (Java, Python, C/++/+-...) como en declarativos (SQL), y si no la tienes, tampoco pasa nada.<br>
Por proponer un pequeño ejemplo para aquellos que hayáis trabajado en SQL, ¿qué hacemos en SQL?<br>
Preguntar por lo que queremos. Desarrollamos una consulta específicando qué, de dónde y bajo qué condiciones queremos obtener el resultado y nos abstraemos totalmente de cómo se devuelve. En Prolog haremos esencialmente lo mismo, dejando todos los puntos necesarios para poder generar la respuesta.<br></p>

</body>
</html>


In [20]:
quizz_1 = [
    {
        'question': '¿Cómo se diferencia la programación imperativa de la declarativa?',
        'options': [
            'La programación imperativa se enfoca en definir qué se quiere lograr,\n mientras que la declarativa se enfoca en cómo lograrlo.',
            'La programación imperativa define el flujo de ejecución paso a paso,\n  mientras que la declarativa se centra en definir las relaciones y el resultado deseado sin controlar el flujo de ejecución.',
            'En la programación imperativa, el motor de ejecución toma decisiones sobre el flujo del programa,\n  mientras que en la declarativa, el programador controla el flujo de ejecución directamente mediante instrucciones.',
            'La programación declarativa se basa en consultas a bases de datos,\n  mientras que la imperativa utiliza estructuras de control como bucles y condicionales.'
        ],
        'correct_option': 'La programación imperativa requiere definir el flujo de ejecución paso a paso,\n  mientras que la declarativa se centra en definir las relaciones y el resultado deseado sin controlar el flujo de ejecución.',
    },
    {
        'question': 'Al hacer sándwiches imperativos, ¿cuál sería el mayor problema que encontramos?',
        'options': [
            'No permite definir tipos de datos específicos para los ingredientes del sándwich.',
            'No contempla excepciones o errores si se proporcionan ingredientes no válidos, lo que puede resultar en un arma biológica penada por la convención de Ginebra.',
            'Especificar incorrectamente la composición del sándwich puede tener consecuencias catastróficas',
            'No es posible comprobar que los ingredientes son correctos, pudiendo preparar sándwiches perjudiciales para la salud.'
        ],
        'correct_option': 'No contempla excepciones o errores si se proporcionan ingredientes no válidos, lo que puede resultar en un arma biológica penada por la convención de Ginebra.',
        'image_path': '../Basico/imagenes/error/error_image_1.png',

    },
    {
        'question': '¿Cuál es una característica destacada de la programación declarativa en Prolog?',
        'options': [
            'Permite al programador definir el flujo de ejecución en detalle mientras define las instrucciones paso a paso.',
            'Se centra en definir las relaciones y reglas del dominio del problema, abstrayéndose del control del flujo de ejecución.',
            'Utiliza un enfoque basado en la definición de pasos secuenciales para alcanzar el resultado deseado.',
            'Permite a los programadores especificar cómo se deben manejar las excepciones y errores durante la ejecución.'
        ],
        'correct_option': 'Se centra en definir las relaciones y reglas del dominio del problema, abstrayéndose del control del flujo de ejecución.',

    },
    {
        'question': '¿Cuáles son las trabas para adentrarse en la programación declarativa y el paradigma lógico?',
        'options': [
            'Pereza.',
            'El acento de los vídeos sobre Prolog es complicadillo de entender.',
            'No tengo la salud mental como para estudiarme un libro sobre Prolog que duplica en peso a los libros de la UNED.',
            'Todas las respuestas anteriores son correctas.'
        ],
        'correct_option': 'Todas las respuestas anteriores son correctas.',
    }
]
run_quiz(quizz_1)


Output()

 <!DOCTYPE html>
<html lang="es">
<body>
    <div class="content">


<h3 id="-siguiente-apartado-sistemas-basados-en-reglas-sbr-ipynb-">- <a href="SBR.ipynb">Siguiente apartado - Sistemas basados en reglas</a></h3>
<h3 id="-apartado-anterior-beneficios-de-prolog-beneficios-ipynb-">- <a href="Beneficios.ipynb">Apartado anterior - Beneficios de Prolog</a></h3>
<h3 id="-volver-al-ndice-indice-ipynb-">- <a href="../Indice.ipynb">Volver al índice</a></h3>



</body>
</html>